**Pre-process Text Data**

In [ ]:
!pip install tqdm

In [1]:
%cd ..
from src.base_model_trainer import BaseModelTrainer
from src.resnet_model_trainer import ResnetHierarchicalModel
from src.metrics import *
from src.losses import *
from src.hierarchy import *
from src.class_imbalance import *
from src.data_generators import *
from src.taxonomy import *
from src.inference_testing import *
%cd -

/home/ec2-user/SageMaker/data-science-product-image
/home/ec2-user/SageMaker/data-science-product-image/notebooks


Using TensorFlow backend.


In [2]:
import pandas as pd

- want images to be downloaded in separate folder in s3
- want to control local image folder 

**Download Model Locally, Recreate Vector, Download Images, Predict New Data**

In [ ]:
## NEW DATA  
data_to_predict_s3_path = 's3://data-science-product-image/datasets/new-data-to-predict/data-input/march_2021_full/'
data_to_predict_s3_file_name = 'march_2021_full.json'
local_json_file_name = data_to_predict_s3_path.split('/')[-2]
print(local_json_file_name)
upload_images_flag = False
s3_images_folder = data_to_predict_s3_path+'images'

## MODEL 
taxonomy_s3_path = 's3://data-science-training-datasets/product-image/mturk-50k-v2/taxonomy.json'
s3_model_artifact = 's3://sagemaker-us-east-1-232188586941/tensorflow-training-2021-03-17-01-56-43-365/output/model.tar.gz'
combination_type = CombinationType.CONCAT
text_cols = 'title,description'


In [26]:
format_new_data(data_to_predict_s3_path, data_to_predict_s3_file_name, local_json_file_name, upload_images_flag)

**Now run the remove-bad-images.ipynb**

In [27]:
from tqdm import tqdm
import glob
from PIL import Image
from PIL.JpegImagePlugin import JpegImageFile

In [28]:
counter = 0
for filename in tqdm(sorted(glob.glob('../files/new_data_to_predict/march_2021_full/*.jpg'))):
    im = Image.open(filename)
    typestr = f'{type(im)}'
    if "'PIL.JpegImagePlugin.JpegImageFile'" not in typestr:
        dest = filename.replace("new_data_to_predict", "exceptions")
        !mv {filename} {dest}

100%|██████████| 435506/435506 [01:36<00:00, 4504.97it/s]


In [29]:
counter

0

**Back to Predicting**

In [18]:
model = download_model_locally(taxonomy_s3_path, s3_model_artifact)

In [19]:
vectorizer = create_vectorizer(combination_type, text_cols)

In [30]:
final_dataset = predict(vectorizer
                    , local_json_file_name
                    , combination_type
                    , model
                    , taxonomy_s3_path)

Found 435506 validated image filenames.
13610/13610 [==============================] - 4243s 312ms/step


In [31]:
final_dataset.head()

,product_id,title,description,confidence,category1,category2,category3,full_category
0,114899844,Take The Day Off Cleansing Balm by Clinique fo...,Free Shipping. Buy Take The Day Off Cleansing ...,0.993884,Hair & Beauty,Skincare,,Hair & Beauty->Skincare
1,162545844,Women's Cass Square Toe Heels - A New Day™,Read reviews and buy Women's Cass Square Toe H...,0.997945,Shoes,Heels,,Shoes->Heels
2,164118817,Easter decor crochet amigurumi pattern - Unico...,Crochet pattern Amigurumi Unicorn toy easter e...,0.846104,Toys & Games,Toys & Games,,Toys & Games->Toys & Games
3,163494839,beautyblender Beautyblender Pro,Beautyblender Pro - beautyblender Pro is made ...,0.901607,Hair & Beauty,Skincare,,Hair & Beauty->Skincare
4,163655943,Hopeless Attraction Dusty Blue Floral Print Ma...,"This dress features a square neckline, smocked...",0.999897,Clothing,Dresses,,Clothing->Dresses


In [32]:
len(final_dataset)

435506

**Write Data to Experimental Zone**

In [33]:
import io
import os
import boto3

local_file_path = 'march_2021.parquet.snappy'
experimental_zone_file_path = 'manual_imports/bi_category_data/march_2021/march_2021.parquet.snappy'

final_dataset.to_parquet(local_file_path)  
    
s3 = boto3.client('s3')
with open(local_file_path, "rb") as f:
    s3.upload_fileobj(f, "prod-rs-datalake-experimental", experimental_zone_file_path, ExtraArgs={'ACL':'bucket-owner-full-control'})
    
os.remove(local_file_path)